In [34]:

!pip install wandb

     |████████████████████████████████| 1.8MB 9.2MB/s 
     |████████████████████████████████| 133kB 24.1MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 163kB 23.4MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=974ea7e0859b516a41caf1c233e5ef8b16e0f27bd53928502b2810f643516a4f
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74841 sha256=928c87590c9f83d7b1f61088292a35547c0c4e964b7ffbd4a0d72e04103a09e5
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=2d228823d743b5312bbbd458476fa534d0558555de6

In [35]:

import wandb
wandb.init()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
print(os.listdir())

['my-try-CNN.ipynb', 'Spokendigit_visualization.ipynb', '.git', 'Spokendigit_Five_features.ipynb', 'README.md', 'Spoken_digit.csv', 'Spoken_digit_X.csv', 'Spokendigit_feature_extraction.ipynb', 'spoken-digit', 'Spokendigit_CNN.ipynb']


In [2]:
os.chdir('drive/MyDrive/ColabNotebooks/speech-recognition/spoken-digits/Spokendigit')

In [1]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
import torchvision
from torchvision.datasets.utils import download_url
import torch.nn as nn
import torch.nn.functional as F
import tarfile
import os
import librosa
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import matplotlib
import csv
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

In [5]:

print(torch.cuda.is_available())

True


In [4]:
digit = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

In [6]:
class SpokenDigit(Dataset):
    def __init__(self, file = None, rootdir = None, transform = None):
        self.df = pd.read_csv(file)
        self.rootdir = rootdir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.loc[i]
        fname, label = row['File'], row['Label']
        ik = self.rootdir+fname[:-4]+'.jpg'
        img = Image.open(ik)
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(digit.index(label))
    
    def getsr(self, i):
        row = self.df.loc[i]
        fname, label = row['File'], row['Label']
        _, sr = librosa.load(self.rootdir+fname)
        return sr

In [7]:
meldset = SpokenDigit('Spoken_digit.csv', 'spoken-digit/', transforms.Compose([transforms.ToTensor()]))

In [8]:
print(len(meldset))

23666


In [9]:
size = len(meldset)
val_size = int(0.1 * size)
train_size = size - val_size 

train_dset, val_dset = random_split(meldset, [train_size, val_size])

train_size, val_size

(21300, 2366)

In [10]:
train_dl = DataLoader(train_dset, 64, True, num_workers=6, pin_memory=True)
val_dl = DataLoader(val_dset, 64, num_workers=6, pin_memory=True)

In [11]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [13]:
device = get_default_device()
device

device(type='cuda')

In [14]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [15]:
class SpokenDigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),

            nn.Flatten(), 
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

    def training_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.cross_entropy(outputs, labels)
        _, pred = torch.max(outputs, 1)
        accuracy = torch.tensor(torch.sum(pred==labels).item()/len(pred))
        return [loss.detach(), accuracy.detach()] 

In [16]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [17]:
def fit(model, train_loader, val_loader, epochs, lr, optimizer_function = torch.optim.Adam):
    history = []
    optimizer = optimizer_function(model.parameters(), lr)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, epochs=epochs, steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        print("Epoch ", epoch)
        #Train
        model.train()
        lrs = []
        tr_loss = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            tr_loss.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            lrs.append(get_lr(optimizer))
            sched.step()
        #Validate
        result = evaluate(model, val_loader)
        result["lrs"] = lrs
        result["train loss"] = torch.stack(tr_loss).mean().item()
        print("Last lr: ", lrs[-1]," Train_loss: ", result["train loss"], " Val_loss: ", result['loss'], " Accuracy: ", result['accuracy'])
        history.append(result)         
    return history

In [22]:
def evaluate(model, loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in loader]
    outputs = torch.tensor(outputs).T
    loss, accuracy = torch.mean(outputs, dim=1)
    return {"loss" : loss.item(), "accuracy" : accuracy.item()}

In [18]:
model = to_device(SpokenDigitModel(), device)
history = []

In [23]:
evaluate(model, val_dl)

{'accuracy': 0.0984906256198883, 'loss': 2.302858591079712}

In [36]:
history.append(fit(model, train_dl, val_dl, 128, 0.001))

Epoch  0



Last lr:  4.159611188916403e-05  Train_loss:  2.2987680435180664  Val_loss:  2.2729859352111816  Accuracy:  0.14582878351211548
Epoch  1



Last lr:  4.6393002654010734e-05  Train_loss:  2.217236042022705  Val_loss:  2.166584014892578  Accuracy:  0.21050839126110077
Epoch  2



Last lr:  5.435859286315734e-05  Train_loss:  2.1086695194244385  Val_loss:  2.0668647289276123  Accuracy:  0.2827620804309845
Epoch  3



Last lr:  6.54395881999855e-05  Train_loss:  2.03354549407959  Val_loss:  2.014005422592163  Accuracy:  0.25994443893432617
Epoch  4


...


Last lr:  0.0009997429617913927  Train_loss:  1.5003968477249146  Val_loss:  1.4911426305770874  Accuracy:  0.9657667279243469
Epoch  38



Last lr:  0.0009998893609390691  Train_loss:  1.4998914003372192  Val_loss:  1.506272792816162  Accuracy:  0.9475942850112915
Epoch  39



Last lr:  0.0009992134106552783  Train_loss:  1.4974396228790283  Val_loss:  1.5027207136154175  Accuracy:  0.9521986246109009
Epoch  40



Last lr:  0.0009979238061351152  Train_loss:  1.4984183311462402  Val_loss:  1.4989490509033203  Accuracy:  0.9530705809593201
Epoch  41


KeyboardInterrupt: ignored

In [37]:
torch.save(model, 'spokendigit_cnn_mel.pth')

In [38]:
loaded_model=torch.load('spokendigit_cnn_mel.pth')

In [41]:
@torch.no_grad()
def predict_dl(model, dl):
    torch.cuda.empty_cache()
    batch_probs = []
    batch_targ = []
    for xb, yb in dl:
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
        batch_targ.append(yb.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    batch_targ = torch.cat(batch_targ)
    return [list(values).index(max(values)) for values in batch_probs], batch_targ

In [42]:
r = evaluate(loaded_model, val_dl)
yp, yt = predict_dl(model, val_dl)
print("Loss: ", r['loss'], "\nAccuracy: ", r['accuracy'], "\nF-score: ", f1_score(yt, yp, average='micro'))

Loss:  1.507399559020996 
Accuracy:  0.9404288530349731 
F-score:  0.9404057480980558
